In [2]:
%cd "/content/drive/My Drive/digikala/q2"

/content/drive/My Drive/digikala/q2


In [3]:
!pip install sentence-transformers hazm 

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import xgboost as xgb

import pandas as pd
import re
from nltk.tokenize import RegexpTokenizer

from spacy.lang.fa import stop_words

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from hazm import word_tokenize, stopwords_list
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, recall_score, confusion_matrix, f1_score, 
                             roc_auc_score, classification_report)

from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.ensemble import RandomForestClassifier

from hazm import stopwords_list

from sklearn.pipeline import Pipeline
# Avoiding warnings
import warnings
import os
########### Prevent Warnings ###########
warnings.filterwarnings(action='ignore')
########### Prevent Warnings ###########

In [50]:
df_train = pd.read_csv("data/train_users.csv")
df_test = pd.read_csv("data/test_users.csv")

df_train['type'] = "train"
df_test['type'] = "test"

df = pd.concat([df_train,df_test], ignore_index=True, sort=False)

df['title_fa_product'] = df['title_fa_product'].astype('category')
df['title_fa_category'] = df['title_fa_category'].astype('category')


df['title_fa_product'] = pd.factorize( df['title_fa_product'].values )[0]
df['title_fa_category'] = pd.factorize( df['title_fa_category'].values )[0]

In [6]:
df.head()

,id,title,comment,advantages,disadvantages,title_fa_product,title_fa_category,is_buyer,verification_status,rate,type
0,0,NaN,مثل بقیه محصولات الکل دار پوست رو خشک نمیکنه,NaN,NaN,0,0,1,1.0,100.0,train
1,1,Redmi 8,با این مبلغ اگه امکانات و ارزش خرید واستون مهم...,"[""رم4\r"",""قیمت نسبت به سایر برندها\r"",""طراحی و...","[""نداشتن هندزفری""]",1,1,1,1.0,80.0,train
2,2,عالی,خوبه فقط کج و کوله بدستم رسید ولی پسرم خیلی خو...,NaN,NaN,2,2,1,1.0,100.0,train
3,3,NaN,در کل خوب بود ولی متاسفانه درب محصول شکسته بود...,NaN,NaN,3,3,1,1.0,70.0,train
4,4,NaN,من که خیلی باهاش حال کردم فقط من که همیشه L می...,NaN,NaN,4,4,1,1.0,72.0,train


In [7]:
df.fillna(" ",inplace=True)

df['title'] = df['title'].astype(str)
df['comment'] = df['comment'].astype(str)
df['advantages'] = df['advantages'].astype(str)
df['disadvantages'] = df['disadvantages'].astype(str)

In [8]:
def preprocess(text):
    pattern = r'[\u0600-\u06FF\u0698\u067E\u0686\u06AF]+'
    tokenizer = RegexpTokenizer(pattern)
    text = tokenizer.tokenize(text)
    text = set(text)
    text = ' '.join(text)
    return text

In [9]:
df['title'] = df['title'].apply(lambda x : preprocess(x))
df['comment'] = df['comment'].apply(lambda x : preprocess(x))
df['advantages'] = df['advantages'].apply(lambda x : preprocess(x))
df['disadvantages'] = df['disadvantages'].apply(lambda x : preprocess(x))

In [10]:
df['text'] = df['title'] + " . " + df['comment'] + " . " + df['advantages'] + " . " + df['disadvantages'] 

In [11]:
df.drop(['title','comment','advantages','disadvantages'],axis=1,inplace=True)

In [12]:
df['words_count'] = df.apply(lambda x : len(word_tokenize(x['text'])),axis=1)

In [13]:
df.head()

,id,title_fa_product,title_fa_category,is_buyer,verification_status,rate,type,text,words_count
0,0,0,0,1,1,100,train,. الکل دار رو بقیه نمیکنه خشک پوست مثل محصولا...,12
1,1,1,1,1,1,80,train,. بیخودی برند واستون ارزش امکانات مبلغ گوشی م...,42
2,2,2,2,1,1,100,train,عالی . شد بیشتر خیلی بسته توجه پسرم رسید کوله ...,23
3,3,3,3,1,1,70,train,. متاسفانه شکسته بسته بود مقداریش شده و خوب م...,18
4,4,4,4,1,1,72,train,. کردم گرفتم میپوشیدم لانگه اندازه خیلی حال ک...,20


In [14]:
columns = ['title_fa_product','title_fa_category','is_buyer','rate','words_count']

for i in columns :
    df = pd.concat([df,pd.get_dummies(df[i], prefix=i)],axis=1)
    df.drop([i],axis=1, inplace=True)

In [15]:
X_train , y_train = df[df['type'] == 'train']['text'].values , df[df['type'] == 'train']['verification_status'].values
X_test = df[df['type'] == 'test']['text'].values

In [16]:
swdl = stopwords_list()
tfidf = TfidfVectorizer(tokenizer=word_tokenize,ngram_range=(1,2),stop_words=swdl,max_df=0.98,min_df=0.0008)

X_train_vector = tfidf.fit_transform(X_train)
X_test_vector = tfidf.transform(X_test)

In [22]:
from scipy.sparse import csr_matrix , hstack , vstack

In [20]:
X_train_features = df[df['type'] == 'train'].drop(['verification_status','id','type','text'],axis=1)
X_train_features = csr_matrix(X_train_features)
X_train_combined = hstack((X_train_vector,X_train_features))

In [18]:
X_test_features = df[df['type'] == 'test'].drop(['verification_status','id','type','text'],axis=1)
X_test_features = csr_matrix(X_test_features)
X_test_combined = hstack((X_test_vector,X_test_features))

In [51]:
y_train = df[df['type'] == 'train']['rate'].values

In [52]:
rf = RandomForestRegressor(max_depth=10, random_state=42)
rf.fit(X_train_combined, y_train.tolist())
y_pred = rf.predict(X_test_combined)

In [44]:
y_train

array([0.0, 1.0], dtype=object)

In [53]:
submission = pd.DataFrame({'id' : df[df['type'] == 'test']['id'].values.tolist() , 'rate':y_pred})

In [54]:
submission.to_csv('output_q3_14.csv',index=False)